### Heroes Of Pymoli Data Analysis

* Although just 14% of the players are female, they have spent more then the male player by average. Female players have spent S4.47 and male players have spent S4.07.


* Players between 35-39 years old have spent more than the other age categories by average, S4.76.


* The top 4 popular items average cost is greater than the total items average cost and the top 5 profitable items also have an averagr cost greater than the total items average cost


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# Input File path
input_file = os.path.join("Resources","purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
df_purchase_data = pd.read_csv(input_file)

# Display first 5 rows
df_purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# Verify number of rows for each column
# Verify if some information is missing
df_purchase_data.count()
# There is no missing information

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

## Player Count

* Total number of players


In [3]:
# Calculate the number of unique SNs (number of players)
player_count = df_purchase_data["SN"].nunique()

# Store information in a Pandas DataFrame
df_player_count = pd.DataFrame({"Total Players" : [player_count]})

df_player_count

,Total Players
0,576


## Purchasing Analysis (Total)

* Number of Unique Items

* Average Purchase Price

* Total Number of Purchases

* Total Revenue


In [4]:
# Calculate the number of unique items
unique_items = df_purchase_data["Item ID"].nunique()

# Calculate average purchase price
avg_purchase_price = df_purchase_data["Price"].mean()

# Calculate the total number of purchases
number_purchases = df_purchase_data["Purchase ID"].nunique()

# Calculate Total Revenue
total_revenue = df_purchase_data["Price"].sum()

# Store information in a Pandas DataFrame
df_purchase_analysis = pd.DataFrame({
                                    "Number of Unique Items" : [unique_items],
                                    "Average Price" : [avg_purchase_price],
                                    "Number of Purchases" : [number_purchases],
                                    "Total Revenue" : [total_revenue]
                                    })

# Formating data
df_purchase_analysis["Average Price"] = df_purchase_analysis["Average Price"].map("${:,.2f}".format)
df_purchase_analysis["Total Revenue"] = df_purchase_analysis["Total Revenue"].map("${:,.2f}".format)

df_purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players

* Percentage and Count of Female Players

* Percentage and Count of Other / Non-Disclosed


In [5]:
# Create Grouped DataFrame
df_gender_demo_grp = df_purchase_data.groupby("Gender")

# Calculate Number of Player by Gender and store in a Pandas DataFrame
df_gender_number_players = pd.DataFrame(df_gender_demo_grp["SN"].nunique())

# Rename column
df_gender_number_players.columns = ["Number of Players"]

# Calculate Percentage of Players
df_gender_number_players["Percentage of Players"] = round(df_gender_number_players["Number of Players"] / player_count * 100,2)

# Sort Dataframe by Total Count
df_gender_number_players.sort_values("Number of Players", ascending = False, inplace = True)

df_gender_number_players

,Number of Players,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* The below each broken by gender

    * Purchase Count
    
    * Average Purchase Price

    * Total Purchase Value
    
    * Average Purchase Total per Person by Gender
    

In [6]:
# Calculate the total number of purchases
df_gender_number_purchases = pd.DataFrame(df_gender_demo_grp["Purchase ID"].nunique())
df_gender_number_purchases.columns = ["Purchase Count"]

# Calculate average purchase price
df_gender_avg_purchase_price = pd.DataFrame(df_gender_demo_grp["Price"].mean())
df_gender_avg_purchase_price.columns = ["Average Purchase Price"]

# Calculate Total Purchase Value
df_gender_total_revenue = pd.DataFrame(df_gender_demo_grp["Price"].sum())
df_gender_total_revenue.columns = ["Total Purchase Value"]

# Calculate Number of Player by Gender
#df_gender_number_players = pd.DataFrame(df_gender_demo_grp["SN"].nunique())
#df_gender_number_players.columns = ["Number of Players"]

# Create a summary Pandas DataFrame
df_gender_purchase_analysis = pd.merge(df_gender_number_purchases,df_gender_avg_purchase_price,on="Gender")
df_gender_purchase_analysis = pd.merge(df_gender_purchase_analysis,df_gender_total_revenue,on="Gender")
df_gender_purchase_analysis = pd.merge(df_gender_purchase_analysis,df_gender_number_players,on="Gender")

# Calculate Average Total Purchase per Person
df_gender_purchase_analysis["Avg Total Purchase per Person"] = df_gender_purchase_analysis["Total Purchase Value"] / df_gender_purchase_analysis["Number of Players"]

# Formating data
df_gender_purchase_analysis["Average Purchase Price"] = df_gender_purchase_analysis["Average Purchase Price"].map("${:,.2f}".format)
df_gender_purchase_analysis["Total Purchase Value"] = df_gender_purchase_analysis["Total Purchase Value"].map("${:,.2f}".format)
df_gender_purchase_analysis["Avg Total Purchase per Person"] = df_gender_purchase_analysis["Avg Total Purchase per Person"].map("${:,.2f}".format)

# Remove "Number of Players" and "Percentage of Players" columns
df_gender_purchase_analysis = df_gender_purchase_analysis.drop(["Number of Players","Percentage of Players"],axis=1)

df_gender_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Create bins for ages

* Categorize the existing players using the age bins.

* Calculate the numbers and percentages by age group

* Create a summary data frame to hold the results


In [7]:
# Verify minimun and maximun ages
print(f'Minimun age: {df_purchase_data["Age"].min()}')
print(f'Maximun age: {df_purchase_data["Age"].max()}')

Minimun age: 7
Maximun age: 45


In [8]:
# Create bins for ages
bins = [0,9,14,19,24,29,34,39,150]
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
df_purchase_data["Age Bins"] = pd.cut(df_purchase_data["Age"], bins = bins, labels = labels)

df_purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bins
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [9]:
# Create grouped DataFrame
df_age_demo_grp = df_purchase_data.groupby('Age Bins')

# Calculate Number of Player by Age and store in a Pandas DataFrame
df_age_number_players = pd.DataFrame(df_age_demo_grp["SN"].nunique())

# Rename column
df_age_number_players.columns = ["Number of Players"]

# Calculate Percentage of Players
df_age_number_players["Percentage of Players"] = round(df_age_number_players["Number of Players"] / player_count * 100,2)

df_age_number_players

,Number of Players,Percentage of Players
Age Bins,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

    * Purchase Count
    
    * Average Purchase Price
    
    * Total Purchase Value
    
    * Average Purchase Total per Person by Age Group


In [10]:
# Calculate the total number of purchases
df_age_number_purchases = pd.DataFrame(df_age_demo_grp["Purchase ID"].nunique())
df_age_number_purchases.columns = ["Purchase Count"]

# Calculate average purchase price
df_age_avg_purchase_price = pd.DataFrame(df_age_demo_grp["Price"].mean())
df_age_avg_purchase_price.columns = ["Average Purchase Price"]

# Calculate Total Purchase Value
df_age_total_revenue = pd.DataFrame(df_age_demo_grp["Price"].sum())
df_age_total_revenue.columns = ["Total Purchase Value"]

# Create a summary Pandas DataFrame
df_age_purchase_analysis = pd.merge(df_age_number_purchases,df_age_avg_purchase_price,on="Age Bins")
df_age_purchase_analysis = pd.merge(df_age_purchase_analysis,df_age_total_revenue,on="Age Bins")
df_age_purchase_analysis = pd.merge(df_age_purchase_analysis,df_age_number_players,on="Age Bins")

# Calculate Average Total Purchase per Person
df_age_purchase_analysis["Avg Total Purchase per Person"] = df_age_purchase_analysis["Total Purchase Value"] / df_age_purchase_analysis["Number of Players"]

# Formating data
df_age_purchase_analysis["Average Purchase Price"] = df_age_purchase_analysis["Average Purchase Price"].map("${:,.2f}".format)
df_age_purchase_analysis["Total Purchase Value"] = df_age_purchase_analysis["Total Purchase Value"].map("${:,.2f}".format)
df_age_purchase_analysis["Avg Total Purchase per Person"] = df_age_purchase_analysis["Avg Total Purchase per Person"].map("${:,.2f}".format)

# Remove "Number of Players" and "Percentage of Players" columns
df_age_purchase_analysis = df_age_purchase_analysis.drop(["Number of Players","Percentage of Players"],axis=1)

df_age_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bins,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value

    * SN
    
    * Purchase Count
    
    * Average Purchase Count
    
    * Total Purchase Value
    

In [11]:
# Create grouped DataFrame
df_sn_purchase_grp = df_purchase_data.groupby("SN")

# Calculate the total number of purchases
df_sn_number_purchases = pd.DataFrame(df_sn_purchase_grp["Purchase ID"].nunique())
df_sn_number_purchases.columns = ["Number of Purchases"]

# Calculate average purchase price
df_sn_avg_purchase_price = pd.DataFrame(df_sn_purchase_grp["Price"].mean())
df_sn_avg_purchase_price.columns = ["Average Purchase Price"]

# Calculate total purchase value
df_sn_total_revenue = pd.DataFrame(df_sn_purchase_grp["Price"].sum())
df_sn_total_revenue.columns = ["Total Purchase Value"]

# Store information in a summary Pandas DataFrame
df_top_spenders = pd.merge(df_sn_number_purchases,df_sn_avg_purchase_price,on="SN")
df_top_spenders = pd.merge(df_top_spenders,df_sn_total_revenue,on="SN")

# Sort Data by Total Purchase Value
df_top_spenders.sort_values(by="Total Purchase Value", ascending = False, inplace = True)

# Formating data
df_top_spenders["Average Purchase Price"] = df_top_spenders["Average Purchase Price"].map("${:,.2f}".format)
df_top_spenders["Total Purchase Value"] = df_top_spenders["Total Purchase Value"].map("${:,.2f}".format)

df_top_spenders.head()

,Number of Purchases,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Identify the 5 most popular items by purchase count

    * Item ID
    
    * Item Name

    * Purchase Count

    * Item Price

    * Total Purchase Value


In [12]:
# Create grouped DataFrame
df_item_purchase_grp = df_purchase_data.groupby(["Item ID","Item Name"])

# Calculate the total number of purchases
df_item_purchase = pd.DataFrame(df_item_purchase_grp["Purchase ID"].nunique())
df_item_purchase.columns = ["Number of Purchases"]

# Calculate average item price
df_item_average_price = pd.DataFrame(df_item_purchase_grp["Price"].mean())
df_item_average_price.columns = ["Average Item Price"]

# Calculate total purchase value
df_item_total_revenue = pd.DataFrame(df_item_purchase_grp["Price"].sum())
df_item_total_revenue.columns = ["Total Purchase Value"]

# Store information in a summary Pandas DataFrame
df_popular_items = pd.merge(df_item_purchase,df_item_average_price,on=["Item ID","Item Name"])
df_popular_items = pd.merge(df_popular_items,df_item_total_revenue,on=["Item ID","Item Name"])

# Sort Data by Number of Purchase
df_popular_items.sort_values(by="Number of Purchases", ascending = False, inplace = True)

# Sort Data by total purchase value and store in a new DataFrame of Most Profitable Items
df_profit_items = df_popular_items.sort_values(by="Total Purchase Value", ascending = False)

# Formating data
df_popular_items["Average Item Price"] = df_popular_items["Average Item Price"].map("${:,.2f}".format)
df_popular_items["Total Purchase Value"] = df_popular_items["Total Purchase Value"].map("${:,.2f}".format)

df_popular_items.head()

,,Number of Purchases,Average Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Identify the 5 most profitable items by total purchase value

    * Item ID
    
    * Item Name

    * Purchase Count

    * Item Price
    
    * Total Purchase Value


In [13]:
# Formating data
df_profit_items["Average Item Price"] = df_profit_items["Average Item Price"].map("${:,.2f}".format)
df_profit_items["Total Purchase Value"] = df_profit_items["Total Purchase Value"].map("${:,.2f}".format)

df_profit_items.head()

,,Number of Purchases,Average Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
